In [4]:
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Flatten
from keras.callbacks import TensorBoard
from datetime import time
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
df = pd.read_csv('imdb_master.csv',encoding='latin-1')
df.head()

,Unnamed: 0,type,review,label,file
0,0,test,Once again Mr. Costner has dragged out a movie...,neg,0_2.txt
1,1,test,This is an example of why the majority of acti...,neg,10000_4.txt
2,2,test,"First of all I hate those moronic rappers, who...",neg,10001_1.txt
3,3,test,Not even the Beatles could write songs everyon...,neg,10002_3.txt
4,4,test,Brass pictures (movies is not a fitting word f...,neg,10003_3.txt


In [4]:
len(df)

100000

In [5]:
df=df[:20000]

In [6]:
sentences = df['review'].values
y = df['label'].values

In [7]:
#tokenizing data
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(sentences)

In [8]:
#getting the vocabulary of data
sentences = tokenizer.texts_to_matrix(sentences)

In [9]:
sentences.shape[1]

2000

In [10]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(sentences, y, test_size=0.25, random_state=1000)

In [11]:
model = Sequential()

model.add(layers.Dense(300,input_dim=sentences.shape[1], activation='relu'))
model.add(layers.Dense(20, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])

In [13]:
tensorboard = TensorBoard(log_dir="logs/", histogram_freq=1, write_graph=True, write_images=False)

In [14]:
history = model.fit(X_train,y_train, epochs=5, verbose=True, validation_data=(X_test,y_test), batch_size=256,callbacks=[tensorboard])
[test_loss, test_acc] = model.evaluate(X_test, y_test)
print("Evaluation result on Test Data : Loss = {}, accuracy = {}".format(test_loss, test_acc))

Train on 15000 samples, validate on 5000 samples
Epoch 1/5
15000/15000 [==============================] - 1s 93us/step - loss: 0.5583 - acc: 0.8035 - val_loss: 0.2975 - val_acc: 0.8774
Epoch 2/5
15000/15000 [==============================] - 1s 64us/step - loss: 0.2545 - acc: 0.9035 - val_loss: 0.2831 - val_acc: 0.8868
Epoch 3/5
15000/15000 [==============================] - 1s 64us/step - loss: 0.2252 - acc: 0.9111 - val_loss: 0.2865 - val_acc: 0.8834
Epoch 4/5
15000/15000 [==============================] - 1s 64us/step - loss: 0.2039 - acc: 0.9213 - val_loss: 0.3119 - val_acc: 0.8730
Epoch 5/5
5000/5000 [==============================] - 0s 38us/step
Evaluation result on Test Data : Loss = 0.3088232529461384, accuracy = 0.8794


In [15]:
history.history

{'val_loss': [0.2974769147872925,
  0.28307024483680726,
  0.28653190774917603,
  0.31193826751708986,
  0.30882324714660647],
 'val_acc': [0.8773999994277955,
  0.8867999994277954,
  0.8833999998092651,
  0.8729999999046326,
  0.8793999992370606],
 'loss': [0.5582705538431804,
  0.254524738963445,
  0.22519725665251414,
  0.2038707525173823,
  0.1875746687412262],
 'acc': [0.8035333334287008,
  0.9034666665077209,
  0.9110666667620341,
  0.9213333335240682,
  0.9261333331108094]}

In [ ]:
#loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# Loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [19]:
predict_test = model.predict_classes(X_test[[34], :])
print("The prediction of the 34th in the predicted dataset is: ", predict_test)
print("The prediction of the 34th in the test dataset is: ", y_test[34])


The prediction of the 34th in the predicted dataset is:  [1]
The prediction of the 34th in the test dataset is:  1


In [36]:
#getting the vocabulary of data
# sentences = tokenizer.texts_to_matrix(sentences)

max_review_len= max([len(s.split()) for s in sentences])
vocab_size= len(tokenizer.word_index)+1
sentences = tokenizer.texts_to_sequences(sentences)
padded_docs= pad_sequences(sentences,maxlen=max_review_len)

In [37]:
padded_docs

array([[  0,   0,   0, ...,  33, 498,   8],
       [  0,   0,   0, ..., 194, 413, 160],
       [  0,   0,   0, ...,   8,   1, 188],
       ...,
       [  0,   0,   0, ..., 235, 170, 471],
       [  0,   0,   0, ...,  40,  32,  69],
       [  0,   0,   0, ...,   5,  97,  14]], dtype=int32)

In [38]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(padded_docs, y, test_size=0.25, random_state=1000)

In [39]:
model = Sequential()

model.add(Embedding(2000, 100, input_length=max_review_len))
model.add(Flatten())
model.add(layers.Dense(300, activation='relu'))
model.add(layers.Dense(20, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])

In [40]:
history = model.fit(X_train,y_train,verbose=1,validation_data=(X_test,y_test),epochs=5)
[test_loss, test_acc] = model.evaluate(X_test, y_test)
print("Evaluation result on Test Data : Loss = {}, accuracy = {}".format(test_loss, test_acc))

Train on 15000 samples, validate on 5000 samples
Epoch 1/5
15000/15000 [==============================] - 484s 32ms/step - loss: 6.0468 - acc: 0.6221 - val_loss: 5.9895 - val_acc: 0.6284
Epoch 2/5
15000/15000 [==============================] - 592s 39ms/step - loss: 6.0626 - acc: 0.6239 - val_loss: 5.9895 - val_acc: 0.6284
Epoch 3/5
15000/15000 [==============================] - 555s 37ms/step - loss: 6.0626 - acc: 0.6239 - val_loss: 5.9895 - val_acc: 0.6284
Epoch 4/5
15000/15000 [==============================] - 561s 37ms/step - loss: 6.0626 - acc: 0.6239 - val_loss: 5.9895 - val_acc: 0.6284
Epoch 5/5
5000/5000 [==============================] - 8s 2ms/step
Evaluation result on Test Data : Loss = 5.989485542297364, accuracy = 0.6284


In [1]:
# # Loss
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()


# # Accuracy

# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

In [1]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train =fetch_20newsgroups(subset='train', shuffle=True)

In [2]:
sentences = newsgroups_train.data
y = newsgroups_train.target

In [5]:
# tokenizing data
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(sentences)
# getting the vocabulary of data
# sentences = tokenizer.texts_to_matrix(sentences)

max_review_len= max([len(s.split()) for s in sentences])
vocab_size= len(tokenizer.word_index)+1
sentences = tokenizer.texts_to_sequences(sentences)
padded_docs= pad_sequences(sentences,maxlen=max_review_len)

le = preprocessing.LabelEncoder()
y = le.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(padded_docs, y, test_size=0.25, random_state=1000)

In [6]:
model = Sequential()
model.add(layers.Dense(300,input_dim=11821, activation='relu'))
model.add(layers.Dense(20, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
history=model.fit(X_train,y_train, epochs=5, verbose=True, validation_data=(X_test,y_test), batch_size=256)
[test_loss, test_acc] = model.evaluate(X_test,y_test)
print("Evaluation result on Test Data : Loss = {}, accuracy = {}".format(test_loss, test_acc))

Train on 8485 samples, validate on 2829 samples
Epoch 1/5
8485/8485 [==============================] - 3s 379us/step - loss: 15.2311 - acc: 0.0534 - val_loss: 15.2185 - val_acc: 0.0555
Epoch 2/5
8485/8485 [==============================] - 2s 286us/step - loss: 15.2289 - acc: 0.0542 - val_loss: 15.1524 - val_acc: 0.0597
Epoch 3/5
8485/8485 [==============================] - 2s 269us/step - loss: 15.2309 - acc: 0.0549 - val_loss: 15.2122 - val_acc: 0.0562
Epoch 4/5
8485/8485 [==============================] - 2s 276us/step - loss: 15.2455 - acc: 0.0541 - val_loss: 15.0926 - val_acc: 0.0636
Epoch 5/5
2829/2829 [==============================] - 0s 92us/step
Evaluation result on Test Data : Loss = 15.021003462224092, accuracy = 0.06786850478517246
